In [ ]:
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model
from keras import backend as K
from keras.callbacks import ModelCheckpoint
import os.path
from keras.preprocessing.image import ImageDataGenerator

def fixed_generator(generator):
    for batch in generator:
        yield (batch, batch)

img_width, img_height = 256, 256

train_data_dir = '/Users/pavelgulaev/Desktop/Диплом/Шок-картинки/train'
validation_data_dir = '/Users/pavelgulaev/Desktop/Диплом/Шок-картинки/valid'
nb_train_samples = 1514
nb_validation_samples = 283
epochs = 50
batch_size = 20

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)
input_img = Input(shape=input_shape)

x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = Model(input_img, decoded)
weights_file="/nfs/home/pgulyaev/cae.best.hdf5"
if (os.path.isfile(weights_file)):
    autoencoder.load_weights(weights_file)
autoencoder.compile(optimizer='adadelta', loss='mse')
checkpoint = ModelCheckpoint(weights_file, verbose=1, save_best_only=True, monitor='val_loss', mode='min')
callbacks_list = [checkpoint]

train_datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode=None)

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode=None)
import sys
oldStdout = sys.stdout
fileLog = open('/Users/pavelgulaev/Desktop/Диплом/cae_logFile', 'w')
sys.stdout = fileLog
autoencoder.fit_generator(
        fixed_generator(train_generator),
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=epochs,
        validation_data=fixed_generator(validation_generator),
        validation_steps=nb_validation_samples // batch_size,
        callbacks=callbacks_list)
sys.stdout = oldStdout